In [42]:
#Importing libraries
import pytesseract
import glob
import pandas as pd
import re
from PIL import Image
import csv

#OCRing images and creating txt files
for item in glob.glob('images/IMG*jpg'):
    print (item)
    #print(pytesseract.image_to_string(Image.open(item))):
    with open('plain-text/'+item[7:15]+'.txt', 'w') as f:
        f.write((pytesseract.image_to_string(Image.open(item))))
        f.close


images/IMG_0563.jpg
images/IMG_0582.jpg
images/IMG_0601.jpg
images/IMG_0604.jpg
images/IMG_0638.jpg


In [133]:
def spouses(name):
    if "Mrs" in name:
        spouse = line.split('Mrs.')
        spouse = spouse[1]
        return spouse[:-2]

In [67]:
# Pulling the professions out of each file
def profession(lines):
    global job
    for line in lines:
        if '.'  in line: 
            if "Mrs." and ')' not in line:
                job = line.split(".")
                break

def profession_couple(lines):
    global job
    global job1
    global job2
    for line in lines:
        if '.'  in line: 
            if "Mrs." and ')' not in line:
                job = line.split(":")
                job1 = job[1].split('.')
                job2= job[2].split('.')
                break



In [68]:
def gender(name):
    global sex
    if 'Mrs' in name: 
        sex = "Female"
        return sex
    elif "wife" in name:
        sex = 'Female'
        return sex
    elif 'Miss' in name:
        sex = 'Female'
        return sex
    elif "Mr" in name:
        sex = 'Male'
        return sex
    else:
        sex = "Male"
        return sex
        


In [138]:
#Pulling out place of origin
def origins(full):
    global origin
    index = 0
    while index < len(full):
        if "from" in full[index]:
            origin_statement = full[index]
            words = origin_statement.split(" ")
            words[:] = [item for item in words if item != '']
            iteration = 0
            while iteration < len(words):
                if 'from' in words[iteration]:
                    if iteration <= len(words) -4:
                        origin = words[iteration+1] + " " + words[iteration+2] + " " + words[iteration+3]
                    else:
                        origin = words[iteration+1:]
                iteration +=1
            break
        elif "From" in full[index]:
            origin_statement = full[index]
            words = origin_statement.split(" ")
            words[:] = [item for item in words if item != '']
            iteration = 0
            while iteration < len(words):
                if 'From' in words[iteration]:
                    if iteration <= len(words) - 4:
                        origin = words[iteration+1] + " " + words[iteration+2] + " " + words[iteration+3]
                    else:
                        origin = words[iteration+1:])
                iteration +=1
            break
        elif "Born" in full[index]:
            if "/" not in full[index]:
                origin_statement = full[index]
                words = origin_statement.split(" ")
                words[:] = [item for item in words if item != '']
                iteration = 0
                while iteration < len(words):
                    if 'Born' in words[iteration]:
                        if iteration <= len(words) - 4:
                            origin = words[iteration+1] + " " + words[iteration+2] + " " + words[iteration+3]
                        else:
                            origin = words[iteration+1:]
                    iteration +=1
        index += 1




SyntaxError: invalid syntax (<ipython-input-138-f8c47d6cef75>, line 29)

In [137]:
#Create dataframe to store extracted data
df = pd.DataFrame([], columns=('Image','Name', 'Profession', 'Origin', 'Spouse', 'Gender', 'Full Text'))



# Pulling the names out of each of the files
for file in glob.glob('plain-text/IMG*'):
    with open(file) as f:
        fulltext = (" ".join(line.strip() for line in f)) 
        full = re.split("(?<!Ft)(?<!Mrs)(?<!Mr)(?<!Appx)(?<!appx)[.]", fulltext)
    with open(file) as f_obj:
        lines = f_obj.readlines()
        for line in lines:
            if ',' in line:
                name = line
                #Getting Spouse's Name
                if "and" in name:
                    origins(full)
                    profession_couple(lines)
                    name1 = name.split('and')[0]
                    spouse = name.split('and')[1]
                    name2 = name1.split(',')[0]+ ','+ name.split('and')[1]
                    dftemp=pd.DataFrame([[file[11:-4],name1[:-1], job1[0], origin, name2[:-1], gender(name1), fulltext]], 
                                      columns =('Image','Name', 'Profession', 'Origin','Spouse', 'Gender', 'Full Text'))
                    df =df.append(dftemp)
                    dftemp=pd.DataFrame([[file[11:-4],name2[:-1], job2[0], origin, name1, gender(name2), fulltext]], 
                                        columns =('Image','Name','Profession', 'Origin','Spouse','Gender', 'Full Text'))
                    df =df.append(dftemp)
                    break
                else:
                    origins(full)
                    profession(lines)
                    dftemp=pd.DataFrame([[file[11:-4],name[:-1],job[0], origin, spouses(name), gender(name), fulltext]], 
                                        columns = ('Image','Name', 'Profession', 'Origin','Spouse', 'Gender', 'Full Text'))
                    df =df.append(dftemp)
                    break

df.to_csv('table.csv')
df

,Image,Name,Profession,Origin,Spouse,Gender,Full Text
0,IMG_0563,"TAYLOR, Richard (Dick)",Operator/floorman/TES monitor,"['Muskogee,', 'Oklahoma']",None,Male,"TAYLOR, Richard (Dick) Operator/floorman/TES ..."
0,IMG_0582,"WAHE, Robert",Floorman,"Cedar Rapids, Iowa,","WAHE, wife (""0."")",Male,"WAHE, Robert and wife (""0."") BOB: Floorman. ""..."
0,IMG_0582,"WAHE, wife (""0."")",Proofreader,"Cedar Rapids, Iowa,","WAHE, Robert",Female,"WAHE, Robert and wife (""0."") BOB: Floorman. ""..."
0,IMG_0601,"YEITER, Robert Marion",Floorman,"Lafay- ette, Ind",None,Male,CONFIDENTIAL INFORMATION FOR ITU REPRESENTATIV...
0,IMG_0604,"rammm,Jmms(Jm0",STEREOTYPER/PRESSMAN,"Ft. Smith, Ark",None,Male,CONFIDENTIAL INFORMATION FOR ITU REPRESENTATIV...
0,IMG_0638,"PRIMER, Joan (Mrs. Donald Premer)",TI‘S operator/proofreader,ZnnesviW strikebreaker Donald,Donald Premer,Female,CONFIDENTIAL INFORMATION FOR ITU REPRESENTATIV...
